In [ ]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 89.5 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

# ----- Step 1: Load Shapefile -----
shapefile_path = "/content/sed/Lithology.shp"  # Update path
gdf = gpd.read_file(shapefile_path)

# ----- Step 2: Reproject to UTM (meters) -----
gdf = gdf.to_crs(epsg=32643)  # Example: UTM zone 43N (adjust based on your area)

# ----- Step 3: Encode Lithology Column -----
label_encoder = LabelEncoder()
gdf["litho_encoded"] = label_encoder.fit_transform(gdf["lithologic"])
scaler = MinMaxScaler()
gdf["litho_norm"] = scaler.fit_transform(gdf[["litho_encoded"]])

# ----- Step 4: Create 30m Grid -----
xmin, ymin, xmax, ymax = gdf.total_bounds
res = 30  # 30 meters
width = int((xmax - xmin) / res)
height = int((ymax - ymin) / res)
transform = from_origin(xmin, ymax, res, res)

# ----- Step 5: Rasterize Lithology -----
shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf["litho_norm"]))
raster = rasterize(
    shapes=shapes,
    out_shape=(height, width),
    transform=transform,
    fill=np.nan,
    dtype='float32'
)

# ----- Step 6: Save GeoTIFF -----
output_tif = "lithology_30m_normalized.tif"
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype='float32',
    crs=gdf.crs,
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(raster, 1)

print("Saved 30m lithology raster:", output_tif)
print("Classes mapped:", list(label_encoder.classes_))


Saved 30m lithology raster: lithology_30m_normalized.tif
Classes mapped: ['Agglomerate', 'Amphibolite', 'Argillite', 'Banded Ferruginous Chert', 'Banded Iron Formation', 'Banded Magnetite Quartzite', 'Banded ferruginous quartzite', 'Chlorite schist', 'Conglomerate', 'Crystalline limestone', 'Diorite', 'Dolerite', 'Dolerite/Gabbro', 'Dolomite', 'Dolomitic limestone', 'Fuchsite quartzite', 'Gabbro', 'Granite', 'Granite gneiss', 'Granitoid', 'Graywacke', 'Grey granite', 'Limestone', 'Manganiferous chert', 'Manganiferous phyllite', 'Meta basalt', 'Meta gabbro', 'Meta rhyolite', 'Meta ultramafite', 'Migmatite gneiss', 'Mylonitised granite', 'Mylonitised granite gneiss', 'Pegmatite', 'Phyllite with bands of dolomite', 'Quartz', 'Quartz vein', 'Quartz-chlorite schist', 'Quartz-kyanite schist', 'Quartzite']
